## Build a News ETL Data Pipeline Using Python and SQLite

### Task 1: Import Libraries and Connect to the News API

In [2]:
import pandas as pd
import sqlite3
import logging
from newsapi import NewsApiClient

### Task 2: Retrieve and Print News Articles

In [3]:
news_api_key = '889a3707d64b4695aabb3a24040c9ab7'

news_api = NewsApiClient(api_key = news_api_key)

In [4]:
def extract_news_data():
    try:
        result = news_api.get_everything(q = "AI", language = 'en', sort_by = 'publishedAt')
        logging.info('Coneection is successful.')
        return result['articles']
    except:
        logging.error('Connection is unsuccessfull')
        return None


articles = extract_news_data()

print(articles[:3])

[{'source': {'id': None, 'name': 'Biztoc.com'}, 'author': 'coingape.com', 'title': 'Cathie Wood’s Ark Invest Declares Its Holdings In ChatGPT’s OpenAI', 'description': 'Cathie Wood’s Ark Investment has been actively betting on the crypto and AI space, and in its latest disclosure, Ark Invest. disclosed its holdings in ChatGPT’s parent firm OpenAI. Ark Invest’s OpenAI Investment Disclosure On Thursday, April 10, Ark Invest in…', 'url': 'https://biztoc.com/x/a2b0841810524302', 'urlToImage': 'https://c.biztoc.com/p/a2b0841810524302/s.webp', 'publishedAt': '2024-04-12T04:54:06Z', 'content': 'Cathie Woods Ark Investment has been actively betting on the crypto and AI space, and in its latest disclosure, Ark Invest. disclosed its holdings in ChatGPTs parent firm OpenAI.Ark Invests OpenAI In… [+287 chars]'}, {'source': {'id': None, 'name': '[Removed]'}, 'author': None, 'title': '[Removed]', 'description': '[Removed]', 'url': 'https://removed.com', 'urlToImage': None, 'publishedAt': '1970-01-01

### Task 3: Clean Author Column

In [5]:
def clean_author_column(text):
    try:
        result = text.split(',')
        return result[0].title()
    except AttributeError:
        return "No Author"


clean_author_column(articles[0])

'No Author'

In [6]:
article = articles[0]
author = article['author']
title = article['title']
publishedAt = article['publishedAt']
content = article['content']
url = article['url']

print(author, title, publishedAt, content, url)

coingape.com Cathie Wood’s Ark Invest Declares Its Holdings In ChatGPT’s OpenAI 2024-04-12T04:54:06Z Cathie Woods Ark Investment has been actively betting on the crypto and AI space, and in its latest disclosure, Ark Invest. disclosed its holdings in ChatGPTs parent firm OpenAI.Ark Invests OpenAI In… [+287 chars] https://biztoc.com/x/a2b0841810524302


### Task 4: Transform News Data

In [7]:
def transform_news_data(articles):
    article_list = []

    for article in articles:
        lis = []
        for key, value in article.items():
            if key in ['author', 'title', 'publishedAt', 'content', 'url', 'source']:
                if key == 'source':
                    lis.append(article.get('name', 0))
                else:
                    lis.append(value)
        
        article_list.append(lis)

    df = pd.DataFrame(article_list, columns= ['Source', 'Author Name', 'News Title', 'URL', 'Date Published', 'Content']) 

    df['Date Published'] = pd.to_datetime(df['Date Published']).dt.strftime('%Y-%m-%d %H:%M:%S')

    df['Author Name'] = df['Author Name'].apply(clean_author_column)

    return df

transformed_data = transform_news_data(articles)

transformed_data      


,Source,Author Name,News Title,URL,Date Published,Content
0,0,Coingape.Com,Cathie Wood’s Ark Invest Declares Its Holdings...,https://biztoc.com/x/a2b0841810524302,2024-04-12 04:54:06,Cathie Woods Ark Investment has been actively ...
1,0,No Author,[Removed],https://removed.com,1970-01-01 00:00:00,[Removed]
2,0,Marisa Chimprabha (World),When does “artificial” stop being so?,https://www.thaipbsworld.com/when-does-artific...,2024-04-12 04:52:04,"Many months are still to go for 2024, but perh..."
3,0,John Callaham,VPN by Google One is shutting down less than f...,https://www.neowin.net/news/vpn-by-google-one-...,2024-04-12 04:52:01,"In October 2020, Google added a new feature as..."
4,0,Daniel Calbimonte,AZ-500 Microsoft Azure Security Exam Study Guide,https://www.mssqltips.com/sqlservertip/7953/az...,2024-04-12 04:50:40,By: Daniel Calbimonte | \r\nUpdated: 2024-0...
...,...,...,...,...,...,...
95,0,Investing.Com,GigaCloud Technology CEO Lei Wu sells shares w...,https://www.investing.com/news/company-news/gi...,2024-04-12 01:43:10,GigaCloud Technology Inc (NASDAQ:GCT) has repo...
96,0,Gigazine（ギガジン）,AppleはAIに最適化した「M4チップ」を2024年中にリリースすることを目指している,https://news.livedoor.com/article/detail/26215...,2024-04-12 01:42:00,AppleAIM42024AppleBloomberg\r\nApple (AAPL) Re...
97,0,Volkmar Richter,"MOVIES: Civil War's extreme US future, Food In...",https://www.nationalobserver.com/2024/04/11/re...,2024-04-12 01:39:53,The film I've been most anticipating is finall...
98,0,Toi Tech Desk,Microsoft to Windows 10 users: It may be time ...,https://timesofindia.indiatimes.com/technology...,2024-04-12 01:35:55,The TOI Tech Desk is a dedicated team of journ...


### Task 5: Load the Data into SQLite Database


In [8]:
def load_news_data(data):
    with sqlite3.connect("/usercode/news_data.sqlite") as connection:
        cursor = connection.cursor()
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS news_table (
                "Source" VARCHAR(30),
                "Author Name" TEXT,
                "News Title" TEXT,
                "URL" TEXT,
                "Date Published" TEXT,
                "Content" TEXT
            )
        ''')
    data.to_sql(name="news_table", con=connection, index=False, if_exists="append")

load_news_data(transformed_data)

### Task 8: Verify Data Loading

In [9]:
with sqlite3.connect("/usercode/etl_news_data.sqlite") as connection:
    df = pd.read_sql("SELECT * FROM news_table;", connection)

df.head()

DatabaseError: Execution failed on sql 'SELECT * FROM news_table;': no such table: news_table